In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [ ]:
df = pd.read_excel('E:/OneDrive/Desktop/Online Retail.xlsx')
print(df.head())
print("\nData Info:")
print(df.info())
print("\nSummary Statistics:")
print(df.describe())
print("\nNumber of Unique Customers:", df['CustomerID'].nunique())


In [ ]:
df.dropna(subset=['CustomerID'], inplace=True)
df['CustomerID'] = df['CustomerID'].astype(int)

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)]

In [ ]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
print("\nCleaned Data Shape:", df.shape)

In [ ]:
latest_date = df['InvoiceDate'].max() + dt.timedelta(days=1)
rfm_df = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (latest_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'Quantity': 'sum',
    'UnitPrice': 'sum'
})
rfm_df.rename(columns={'InvoiceDate': 'Recency',
                       'InvoiceNo': 'Frequency',
                       'Quantity': 'TotalQuantity',
                       'UnitPrice': 'TotalRevenue'}, inplace=True)

rfm_df['Monetary'] = rfm_df['TotalQuantity'] * rfm_df['TotalRevenue'] 

rfm_df = rfm_df[['Recency', 'Frequency', 'Monetary']]

print("\nRFM Data:")
print(rfm_df.head())


In [ ]:
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
sns.histplot(rfm_df['Recency'])
plt.title('Recency Distribution')
plt.subplot(1, 3, 2)
sns.histplot(rfm_df['Frequency'])
plt.title('Frequency Distribution')
plt.subplot(1, 3, 3)
sns.histplot(rfm_df['Monetary'])
plt.title('Monetary Distribution')
plt.tight_layout()
plt.show()

In [ ]:
rfm_log = np.log1p(rfm_df)
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
sns.histplot(rfm_log['Recency'])
plt.title('Log-transformed Recency')
plt.subplot(1, 3, 2)
sns.histplot(rfm_log['Frequency'])
plt.title('Log-transformed Frequency')
plt.subplot(1, 3, 3)
sns.histplot(rfm_log['Monetary'])
plt.title('Log-transformed Monetary')
plt.tight_layout()
plt.show()

In [ ]:
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm_log)
rfm_scaled_df = pd.DataFrame(rfm_scaled, index=rfm_log.index, columns=rfm_log.columns)
print("\nScaled RFM Data:")
print(rfm_scaled_df.head())

In [ ]:
inertia = []
range_n_clusters = range(2, 11)
for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    kmeans.fit(rfm_scaled_df)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range_n_clusters, inertia, marker='o', linestyle='-')
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.xticks(range_n_clusters)
plt.grid(True)
plt.show()

In [ ]:
silhouette_scores = []
for n_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(rfm_scaled_df)
    silhouette_avg = silhouette_score(rfm_scaled_df, cluster_labels)
    silhouette_scores.append(silhouette_avg)

plt.figure(figsize=(10, 6))
plt.plot(range_n_clusters, silhouette_scores, marker='o', linestyle='-')
plt.title('Silhouette Analysis for Optimal k')
plt.xlabel('Number of Clusters')
plt.ylabel('Silhouette Score')
plt.xticks(range_n_clusters)
plt.grid(True)
plt.show()

In [ ]:
n_clusters = 3  # Choose your optimal number of clusters
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
rfm_df['Cluster'] = kmeans.fit_predict(rfm_scaled_df)

print("\nRFM Data with Cluster Labels:")
print(rfm_df.head())

In [ ]:
cluster_analysis = rfm_df.groupby('Cluster').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
}).round(1)
print("\nCluster Analysis:")
print(cluster_analysis)

def assign_segment(row):
    if row['Cluster'] == 0:
        return 'Loyal Customers'
    elif row['Cluster'] == 1:
        return 'Potential High-Value'
    else:
        return 'At-Risk Customers' # Adjust labels based on your cluster analysis

rfm_df['Segment'] = rfm_df.apply(assign_segment, axis=1)
print("\nRFM Data with Segments:")
print(rfm_df.head())

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(data=rfm_df, x='Segment', palette='viridis')
plt.title('Customer Segment Distribution')
plt.xlabel('Customer Segment')
plt.ylabel('Number of Customers')
plt.show()

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
sns.boxplot(data=rfm_df, x='Segment', y='Recency', palette='viridis')
plt.title('Recency by Segment')
plt.subplot(1, 3, 2)
sns.boxplot(data=rfm_df, x='Segment', y='Frequency', palette='viridis')
plt.title('Frequency by Segment')
plt.subplot(1, 3, 3)
sns.boxplot(data=rfm_df, x='Segment', y='Monetary', palette='viridis')
plt.title('Monetary Value by Segment')
plt.tight_layout()
plt.show()

In [ ]:
print("\nRecommendations for Targeted Marketing:")
print(rfm_df.groupby('Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'Segment': 'count'
}))
print("\nBased on the analysis:")
print("- **Loyal Customers:** High Recency, Frequency, and Monetary values. Focus on retention strategies, loyalty programs, and upselling.")
print("- **Potential High-Value:** Recent purchases, decent frequency and monetary value. Engage them with personalized offers and build long-term relationships.")
print("- **At-Risk Customers:** High Recency, lower Frequency and Monetary values. Implement reactivation campaigns with attractive discounts or new product highlights.")